In [3]:
from baselines.macbf import *

# Evaluate Static

In [4]:
from tqdm import tqdm
import gc
from copy import deepcopy
import torch
import numpy as np
from torch import nn
import math
from models import *   
from core import generate_default_model_name
import pickle as pkl
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

SAVE_GIF = False
ONLY_SHOW_COLLIDE = True
VERBOSE = False

dataset_mode = 'static'

for density in [0,5,10,15,20]:

    with open(f'dataset/{project_name}_{dataset_mode}_{density}.pkl', 'rb') as f:
        valid_dataset = pkl.load(f)

    bnn = create_network()
    bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
    bnn.eval();

    import copy

    collideds = []
    dones = []
    lengths = []
    
    paths = []
    
    path = f'gifs/0513/{project_name}_{version_name}/{density}'
    os.makedirs(path, exist_ok=True)

    for v_idx in tqdm(range(len(valid_dataset))):
        data = valid_dataset[v_idx]
        env = create_env(num_agents=3, size=4, max_dist=1, density=0)
        env.world.obstacles, env.world.agent_goals, env.world.agents = deepcopy(data)
        if project_name=='drone':
            env.world.agents[:, 3:6] = 0
        if SAVE_GIF:
            gif_file = f'gifs/0513/{project_name}_{version_name}/{density}/'+str(v_idx)+f'_decompose_lie.gif'
        else:
            gif_file = None
        collided, done, path = infer(env,bnn,verbose=VERBOSE,n_action=10000,
                                     max_episode_length=512,
                                     spatial_prop=False,lie_derive_safe=False,decompose='random',
                                     stop_at_collision=False,need_gif=gif_file)
        collideds.append(collided)
        dones.append(done)
        lengths.append(len(path))
        paths.append(path)

    print(density, np.any(collideds, axis=-1).mean(), np.mean(collideds), np.mean(dones), np.mean(lengths))

    with open(f'results/{project_name}_{version_name}_{dataset_mode}_{density}.pkl', 'wb') as f:
        pkl.dump({'collideds': collideds,
                  'dones': dones,
                  'lengths': lengths,
                  'paths': paths}, f)

100%|██████████| 100/100 [00:21<00:00,  4.73it/s]


0 0.12 0.08333333333333333 1.0 39.16


100%|██████████| 100/100 [00:42<00:00,  2.34it/s]


5 0.96 0.6533333333333333 1.0 67.13


100%|██████████| 100/100 [00:49<00:00,  2.03it/s]


10 0.98 0.7066666666666667 0.99 76.67


100%|██████████| 100/100 [00:47<00:00,  2.12it/s]


15 0.98 0.7566666666666667 1.0 74.41


100%|██████████| 100/100 [00:51<00:00,  1.93it/s]

20 0.99 0.77 0.99 82.0


# Evaluate Dynamic

In [6]:
from tqdm import tqdm
import gc
from copy import deepcopy
import torch
import numpy as np
from torch import nn
import math
from models import *   
from core import generate_default_model_name
import pickle as pkl
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

for method_id in [1,2]:
    if method_id == 1:
        from baselines.macbf import *
    elif method_id == 2:
        from baselines.ddpg import *
    elif method_id == 3:
        from drone_v34 import *

    dataset_mode = 'dynamic'

    for density in [1,3]+list(2**np.arange(3, 9)):  # 

        with open(f'dataset/{project_name}_{dataset_mode}_{density}.pkl', 'rb') as f:
            valid_dataset = pkl.load(f)

        bnn = create_network()
        bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
        bnn.eval();

        import copy

        collideds = []
        dones = []
        lengths = []

        paths = []

        path = f'gifs/0513/{project_name}_{version_name}/{density}'
        os.makedirs(path, exist_ok=True)

        for v_idx in tqdm(range(len(valid_dataset))):
            data = valid_dataset[v_idx]
            env = create_env(num_agents=density, size = int((density*16)**0.5), max_dist=1, density=0)
            env.world.obstacles, env.world.agent_goals, env.world.agents = deepcopy(data)
            if project_name=='drone':
                env.world.agents[:, 3:6] = 0
            gif_file = None
            collided, done, path = infer(env,bnn,seed=v_idx,bnn=bnn,verbose=VERBOSE,n_action=10000,
                                         max_episode_length=512,
                                         spatial_prop=False,lie_derive_safe=False,decompose='random',
                                         stop_at_collision=False,need_gif=gif_file)
            collideds.append(collided)
            dones.append(done)
            lengths.append(len(path))
            paths.append(path)

        print(density, np.any(collideds, axis=-1).mean(), np.mean(collideds), np.mean(dones), np.mean(lengths))

        with open(f'results/{project_name}_{version_name}_{dataset_mode}_{density}.pkl', 'wb') as f:
            pkl.dump({'collideds': collideds,
                      'dones': dones,
                      'lengths': lengths,
                      'paths': paths}, f)

100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


1 0.67 0.67 1.0 58.1


100%|██████████| 100/100 [00:58<00:00,  1.70it/s]


3 1.0 0.7933333333333333 1.0 94.57


100%|██████████| 100/100 [01:55<00:00,  1.16s/it]


...

64 1.0 0.455625 0.0 513.0


100%|██████████| 100/100 [26:07<00:00, 15.68s/it]


128 1.0 0.472265625 0.0 513.0


100%|██████████| 100/100 [49:20<00:00, 29.61s/it]


256 1.0 0.49328125 0.0 513.0


# Evaluate Mixed

In [4]:
from tqdm import tqdm
import gc
from copy import deepcopy
import torch
import numpy as np
from torch import nn
import math
from models import *   
from core import generate_default_model_name
import pickle as pkl
import os
from baselines.ddpg import *

if project_name == 'multi_dynamic':
    project_name = 'multi_dynamic_dubins'

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

size = 32
dataset_mode = 'mixed'
decompose = None

for density in list([1,3]+list(2**np.arange(3, 10)))[-1:]:

    with open(f'dataset/{project_name}_{dataset_mode}_{density}.pkl', 'rb') as f:
        valid_dataset = pkl.load(f)

    bnn = create_network()
    bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
    bnn.eval();

    import copy

    collideds = []
    dones = []
    lengths = []
    
    paths = []
    
    path = f'gifs/0513/{project_name}_{version_name}/{density}'
    os.makedirs(path, exist_ok=True)

    for v_idx in tqdm(range(len(valid_dataset))):
        data = valid_dataset[v_idx]
        env = create_env(num_agents=density, size = size, max_dist=1, density=0)
        env.world.obstacles, env.world.agent_goals, env.world.agents = deepcopy(data)
        if project_name=='drone':
            env.world.agents[:, 3:6] = 0
        gif_file = None
        collided, done, path = infer(env,bnn,verbose=False,n_action=2000,
                                     max_episode_length=512,
                                     spatial_prop=False,lie_derive_safe=False,decompose=decompose,
                                     stop_at_collision=False,need_gif=gif_file)
        collideds.append(collided)
        dones.append(done)
        lengths.append(len(path))
        paths.append(path)

    print(density, np.any(collideds, axis=-1).mean(), np.mean(collideds), np.mean(dones), np.mean(lengths))

    suffix = '' if decompose=='random' else 'no_decompose_'
    with open(f'results/{project_name}_{version_name}_{suffix}{dataset_mode}_{density}.pkl', 'wb') as f:
        pkl.dump({'collideds': collideds,
                  'dones': dones,
                  'lengths': lengths,
                  'paths': paths}, f)

multi_dynamic


  0%|          | 0/100 [00:40<?, ?it/s]

KeyboardInterrupt



In [6]:
from tqdm import tqdm
import gc
from copy import deepcopy
import torch
import numpy as np
from torch import nn
import math
from models import *   
from core import generate_default_model_name
import pickle as pkl
import os
# from v109 import *
from v0_multi_dynamic_dubins import *
from baselines.ddpg import *

if project_name == 'multi_dynamic':
    project_name = 'multi_dynamic_dubins'

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

size = 32
dataset_mode = 'mixed'
decompose = 'random'

for density in [1,3]+list(2**np.arange(3, 10)):

    with open(f'dataset/{project_name}_{dataset_mode}_{density}.pkl', 'rb') as f:
        valid_dataset = pkl.load(f)

    bnn = create_network()
    bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
    bnn.eval();

    import copy

    collideds = []
    dones = []
    lengths = []
    
    paths = []
    
    path = f'gifs/0513/{project_name}_{version_name}/{density}'
    os.makedirs(path, exist_ok=True)

    for v_idx in tqdm(range(len(valid_dataset))):
        data = valid_dataset[v_idx]
        env = create_env(num_agents=density, size = size, max_dist=1, density=0)
        env.world.obstacles, env.world.agent_goals, env.world.agents = deepcopy(data)
        if project_name=='drone':
            env.world.agents[:, 3:6] = 0
        gif_file = None
        collided, done, path = infer(env,bnn,verbose=False,n_action=10000,
                                     max_episode_length=512,
                                     spatial_prop=False,lie_derive_safe=False,decompose=decompose,
                                     stop_at_collision=False,need_gif=gif_file)
        collideds.append(collided)
        dones.append(done)
        lengths.append(len(path))
        paths.append(path)

    print(density, np.any(collideds, axis=-1).mean(), np.mean(collideds), np.mean(dones), np.mean(lengths))

    suffix = '' if decompose=='random' else 'no_decompose_'
    with open(f'results/{project_name}_{version_name}_{suffix}{dataset_mode}_{density}.pkl', 'wb') as f:
        pkl.dump({'collideds': collideds,
                  'dones': dones,
                  'lengths': lengths,
                  'paths': paths}, f)

100%|██████████| 100/100 [02:08<00:00,  1.29s/it]


1 0.03 0.03 0.85 153.64


100%|██████████| 100/100 [02:52<00:00,  1.73s/it]


3 0.15 0.056666666666666664 0.63 280.05


100%|██████████| 100/100 [05:02<00:00,  3.02s/it]


...

128 1.0 0.105390625 0.0 513.0


100%|██████████| 100/100 [54:58<00:00, 32.99s/it]


256 1.0 0.155390625 0.0 513.0


100%|██████████| 100/100 [1:40:46<00:00, 60.47s/it]


512 1.0 0.265703125 0.0 513.0


In [4]:
!ls

 0217_grid_Q.json2
 0512.txt
 1.txt
 1model_DroneEnv.txt
 1model_DubinsCarEnv_v78.txt
 Certificate_Exploration.ipynb
 Certificate_OneModel-Pure.ipynb
 Certificate_OneModel_Warmup.ipynb
 Inference_Barrier_Drone.ipynb
 Inference_Barrier_Drone_Decompose.ipynb
 Inference_Barrier_Dubins.ipynb
 Inference_Barrier_Dubins_Decompose.ipynb
 Inference_Barrier_Dubins_old.ipynb
 Inference_Barrier_MultiPoint.ipynb
 Inference_Barrier_Point.ipynb
 Inference_Barrier_PointEnv.ipynb
 Inference_Barrier_Swimmer.ipynb
 Inference_Barrier_UR5Env.ipynb
 Inference_DynamicDubins.ipynb
 Inference_Lya_Drone.ipynb
 Q_vs_CLF.ipynb
 Train_Barrier_Dubins.ipynb
 Train_Barrier_MultiPoint-FixObstacle.ipynb
 Train_Barrier_MultiPoint.ipynb
 __init__.py
 __pycache__
 a.png
 arm_env.py
 arm_env2.py
 arrow_boundary.pdf
 azhe1.pdf
 azhe2.pdf
 baselines
 calculate_stats.py
 cam.gif
 chasing.txt
 check_rl_swimmer.ipynb
 configs
 core.py
 count_experience.ipynb
 count_subgraph.ipynb
 dataset
 dataset_copy
 drone.gif
 drone_env.py


# Generate An Example Environment

In [4]:
env = create_env(num_agents=2048, size=64, max_dist=16, density=10)
print('----------env created-----------')
if project_name=='drone':
    env.world.agents[:, 3:6] = 0
gif_file = None
collided, done, path = infer(env,bnn,verbose=False,n_action=10000,
                             max_episode_length=1024,
                             spatial_prop=False,lie_derive_safe=False,decompose='random',
                             stop_at_collision=False,need_gif=gif_file)

----------env created-----------


In [9]:
with open('results/drone_2048.pkl', 'wb') as f:
    pkl.dump([env.world.obstacles, env.world.agent_goals, path], f)

# Chasing Games

In [87]:
from drone_v34 import *
import os

@torch.no_grad()
def infer_chase(env, bnn, threshold=None, max_episode_length=256, 
          n_action=None,
          verbose=False, seed=0, stop_at_collision=False, 
          spatial_prop=None, need_gif=None, 
          decompose=None, lie_derive_safe=None,
          only_show_collide=False):
    
    if spatial_prop is None:
        spatial_prop = SPATIAL_PROP
        
    if n_action is None:
        n_action = n_candidates
        
    if lie_derive_safe is None:
        lie_derive_safe = LIE_DERIVE_SAFE
    
    if threshold is None:
        threshold=THRESHOLD
    if verbose:
        print('----------------------------------------')
        
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    goal_agent_idx = (np.array(range(env.num_agents))-1)%env.num_agents
    dist_goal = np.array(env.world.agents[:,:env.space_dim]-env.world.agents[goal_agent_idx,:env.space_dim])
    dist_goal = np.linalg.norm(dist_goal, axis=-1)
    paths = [deepcopy(env.world.agents)]
    total_trans=0; n_danger=0; 
    no_feasible=0; collided=np.zeros(env.num_agents).astype(bool); thresholds=np.array([threshold]*env.num_agents)
    reward = 0
    
    while True:  
        env.world.agent_goals[:,:env.space_dim] = env.world.agents[goal_agent_idx,:env.space_dim]        
        
        a, v, feasibles, evil_agents = choose_action(bnn=bnn, env=env, explore_eps=0, 
                                                     nominal_eps=0, 
                                                     spatial_prop=spatial_prop, 
                                                     thresholds=thresholds,
                                                     n_action=n_action,
                                                     decompose=decompose)
        next_o, rw, done, info = env.step(a, obs_config=OBS_CONFIG)
        env.world.agent_goals[:,:env.space_dim] = env.world.agents[goal_agent_idx,:env.space_dim]   
        
        prev_danger = info['prev_danger'].data.cpu().numpy().astype(bool)
        next_danger = info['next_danger'].data.cpu().numpy().astype(bool)
        if np.any(next_danger):
            collided = collided | next_danger
            n_danger = n_danger + np.sum(next_danger)
        if verbose:
            print(total_trans, v.min(axis=-1), v.max(axis=-1), np.where(v<=thresholds), np.where(next_danger), evil_agents)
            
        total_trans += 1
        if lie_derive_safe:
            thresholds = 0.9*v+1e-2
        dist_goal_current = np.array(env.world.agents[:,:env.space_dim]-env.world.agents[goal_agent_idx,:env.space_dim])
        dist_goal_current = np.linalg.norm(dist_goal_current, axis=-1)
        reward += (dist_goal - dist_goal_current).clip(0, 2).mean()
        reward -= next_danger.sum() / env.num_agents
        dist_goal = dist_goal_current
        
        paths.append(deepcopy(env.world.agents))

        if np.any(next_danger) and stop_at_collision:
            break        
        
        if done or (total_trans >= max_episode_length):
            break
            
    if (need_gif is not None):
        if (not only_show_collide) or (np.any(collided)):
            env.save_fig(paths, 1000*np.ones((env.num_agents, env.state_dim)), env.world.obstacles, need_gif[:-4]+'_'+str(np.any(collided))+'_'+str(done)+need_gif[-4:])

    safety_rate = 1 -(n_danger / (total_trans * env.num_agents))
            
    return collided, done, safety_rate, reward, paths

In [88]:
# drone
from drone_v34 import *
bnn = create_network()
bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
bnn.eval()

collideds = []
dones = []
lengths = []
paths = []
path = f'gifs/0604/drone/chasing.mp4'
os.makedirs(path, exist_ok=True)


for decompose in ['random', None]:
    for num_agents in [3,16,64,256]:

        safety_rates = []
        rewards = []
        for seed in tqdm([0, 100, 200, 300, 400, 500, 600, 700, 800, 900]):
            torch.manual_seed(seed)
            random.seed(seed)
            np.random.seed(seed)
            env = create_env(num_agents=num_agents, size=max(3, int(2*num_agents**0.5)), min_dist=0, max_dist=2000)
            env.world.agents[:,6:] = 0
            collided, done, safety_rate, reward, paths = infer_chase(env,bnn,verbose=False,n_action=2000,
                  max_episode_length=256, spatial_prop=False, seed=seed,
                  lie_derive_safe=False, decompose=decompose, stop_at_collision=False,need_gif=None)
            safety_rates.append(safety_rate)
            rewards.append(reward)
        with open('chasing.txt', 'a') as f:
            f.write(str(('drone', decompose, num_agents, np.mean(safety_rates), np.mean(rewards), np.std(safety_rates), np.std(rewards))).strip('()')+'\n')

100%|██████████| 10/10 [05:19<00:00, 31.95s/it]


In [89]:
# car
from v109 import *
bnn = create_network()
bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
bnn.eval()

collideds = []
dones = []
lengths = []
paths = []
path = f'gifs/0604/drone/chasing.mp4'
os.makedirs(path, exist_ok=True)

for decompose in ['random', None]:
    for num_agents in [3,16,64,256]:

        safety_rates = []
        rewards = []
        for seed in tqdm([0, 100, 200, 300, 400, 500, 600, 700, 800, 900]):
            torch.manual_seed(seed)
            random.seed(seed)
            np.random.seed(seed)
            env = create_env(num_agents=num_agents, size=max(3, int(num_agents**0.5)), min_dist=2, max_dist=10000)
            collided, done, safety_rate, reward, paths = infer_chase(env,bnn,verbose=False,n_action=2000,
                  max_episode_length=256, spatial_prop=False, seed=seed,
                  lie_derive_safe=False, decompose=decompose, stop_at_collision=False,need_gif=None)
            safety_rates.append(safety_rate)
            rewards.append(reward)
        with open('chasing.txt', 'a') as f:
            f.write(str(('car', decompose, num_agents, np.mean(safety_rates), np.mean(rewards), np.std(safety_rates), np.std(rewards))).strip('()')+'\n')

100%|██████████| 10/10 [07:54<00:00, 47.43s/it]
